In [107]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

COM5 - USB-SERIAL CH340 (COM5)
COM4 - USB Serial Port (COM4)


In [113]:
import serial
import time
ser = serial.Serial('COM5', 115200, timeout=10)

In [151]:
ser.close()

In [4]:
#test echo (lab2)
#ser.write(b'ABCD EFGHI J')
#time.sleep(.001)
#print(ser.read(12))
#works again after ser close + reset on pynq -> check yusuf testbench for mismatch rx and tx

In [135]:
#test mp adder (lab3) (problem: received missing first bit + small bit flips)
#GENERATE RANDOM OPERANDS
import random
import time

ser.reset_input_buffer()
ser.reset_output_buffer()

OPERAND_WIDTH = 512

N_BYTES = int(OPERAND_WIDTH/8)

A = random.randrange(2**(OPERAND_WIDTH-1), 2**(OPERAND_WIDTH)-1)
B = random.randrange(2**(OPERAND_WIDTH-1), 2**(OPERAND_WIDTH)-1)

res = A + B

print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res))
#A_bytes = bytearray.fromhex(format(A, 'x'))
#B_bytes = bytearray.fromhex(format(B, 'x'))
A_bytes = bytearray.fromhex(format(A, '0{}x'.format(N_BYTES * 2)))
B_bytes = bytearray.fromhex(format(B, '0{}x'.format(N_BYTES * 2)))

A     =  0xfd67fb8995f71a4f0016f2ad3384c532b70e36976907bd1913ec118953a4527bbde79c6432eadeec21ae034f9b8148ac1751edbf8d85ddcad78c8cfc4fe7351f
B     =  0xf908f57fce52223a568204a2819d7f8f9e554513217220396f617e52d045a51a7a8b4b4b758419727e16fe8b8350b3f7aece9ae52afca80454d606822caeb306
A + B =  0x1f670f10964493c895698f74fb52244c255637baa8a79dd52834d8fdc23e9f7963872e7afa86ef85e9fc501db1ed1fca3c62088a4b88285cf2c62937e7c95e825


In [143]:
A = 2**(OPERAND_WIDTH-1)+1000
B = 2**(OPERAND_WIDTH)-1000000
res = A + B
print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res))

A     =  0x800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003e8
B     =  0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff0bdc0
A + B =  0x17ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff0c1a8


In [149]:
#SEND THE OPERANDS, READ RESULT and COMPARE
#A_bytes = bytearray.fromhex(format(A, 'x'))
#B_bytes = bytearray.fromhex(format(B, 'x'))
A_bytes = bytearray.fromhex(format(A, '0{}x'.format(N_BYTES * 2)))
B_bytes = bytearray.fromhex(format(B, '0{}x'.format(N_BYTES * 2)))
#SEND OPERANDS
for op_byte in A_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

for op_byte in B_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

time.sleep(0.1)

res_rcvd = ser.read(N_BYTES+1)

res2 = int.from_bytes(res_rcvd, "big")

if res2==res:
    print("Result received correctly!")
else:
    print("Result INCORRECT!")
    
hex_expected = hex(res)[2:].zfill(8)
hex_received = hex(res2)[2:].zfill(8)

print("Hex Expected = ", hex_expected)
print("Hex Received = ", hex_received)

    # Highlight mismatches with '^'
diff_string = ''.join(
        '^' if a != b else ' ' for a, b in zip(hex_expected, hex_received)
    )

print("Difference   = ", diff_string)
print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res))

Result received correctly!
Hex Expected =  17ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff0c1a8
Hex Received =  17ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff0c1a8
Difference   =                                                                                                                                   
A     =  0x800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003e8
B     =  0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff0bdc0
A + B =  0x17ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff0c1a8
